In [10]:
import torch
from torch import nn
import numpy as np
import random
from collections import deque
from itertools import count
import torch.nn.functional as F
from mdp import TradeExecutionEnv, DiscreteTradeSizeWrapper
from tensorboardX import SummaryWriter
from torch.distributions import Categorical


SEED = 42
HORIZON = 5 * 12 * 8
#HORIZON = 10
UNITS_TO_SELL = 40
BATCH_SIZE = 32
EPOCHS = 1000
EPSILON = 0.5
ALPHA = 0.1
GAMMA = 0.99
TEMPERATURE = 0.1

env = TradeExecutionEnv()

trade_sizes = {
  0: 0,
  1: 1,
  2: 2,
  3: 4,
  4: 8,
  #5: 16,
  #6: 32,
  #7: 64,
  #8: 128,
  #9: 250
}
env = DiscreteTradeSizeWrapper(env, trade_sizes)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


class CategoricalPolicy(nn.Module):
    def __init__(self, num_states, num_actions, hidden_dim=32) -> None:
        super().__init__()
        self.fc1 = nn.Linear(num_states, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_actions)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=1)
        return x

    def select_action(self, state):
        probs = self.forward(state)
        m = Categorical(probs)
        action = m.sample()
        return action.item(), m.log_prob(action)

In [11]:
def parse_state(state):
    data = torch.FloatTensor(np.stack([
        state["low"].to_numpy(),
        state["high"].to_numpy(),
        state["close"].to_numpy(),
        state["open"].to_numpy(),
        state["volume"].to_numpy(),
    ])).T
    return torch.concat([
        data,
        torch.repeat_interleave(torch.FloatTensor([[state["units_sold"]]]), 6, 0),
        torch.repeat_interleave(torch.FloatTensor([[state["cost_basis"]]]), 6, 0),
        torch.repeat_interleave(torch.FloatTensor([[state["steps_left"]]]), 6, 0),
    ], dim=1)[-1,:].unsqueeze(0)

In [12]:
def sample_rollouts(policy, env, num_rollouts):
    trajs = []
    for _ in range(num_rollouts):
        tau = []
        state = env.reset(UNITS_TO_SELL, HORIZON, seed=SEED)
        state = parse_state(state)
        done = False
        while not done:
            with torch.no_grad():
                action, _ = policy.select_action(state)
            next_state, reward, done, _, _ = env.step(action)
            next_state = parse_state(next_state)
            tau.append((state, action, reward))
            state = next_state
        states, actions, rewards = zip(*tau)
        states = torch.cat(states)
        actions = torch.tensor([actions])
        rewards = torch.tensor([rewards])
        trajs.append((states, actions, rewards))
    return trajs

In [13]:
def R_tau(r):
    gammas = torch.tensor([GAMMA**i for i in range(len(r))])
    return torch.sum(gammas * r)

def avg_batch_rewards(trajs):
    return torch.mean(torch.stack([R_tau(r) for _, _, r in trajs]))

def grad_log_pi(policy, states, actions):
    logits = policy(states)
    log_probs = torch.log(logits)
    log_probs = log_probs.gather(1,actions).T.flatten()
    g = [torch.autograd.grad(log_p, policy.parameters(), retain_graph=True) for log_p in log_probs]
    return g

In [14]:
def grad_log_tau(policy, tau):
  g = grad_log_pi(policy, tau[0], tau[1])
  stacked_g = [torch.stack(g_).view(len(tau[0]), -1) for g_ in zip(*g)]
  return torch.cat(stacked_g, axis=1).sum(axis=0)

def grad_U_tau(policy, tau):
  return grad_log_tau(policy, tau) * R_tau(tau[2])


In [15]:
def surrogate_objective(policy, new_policy, trajs):
    means = []
    for tau in trajs:
      states, actions, rewards = tau
      prob_ratios = new_policy(states).gather(1, actions).T.flatten() / policy(states).gather(1, actions).T.flatten()
      discounted_rewards = torch.FloatTensor([torch.sum(torch.stack([rewards.T[j] * GAMMA ** (j-i) for j in range(i,len(rewards.T))])) for i in range(len(rewards.T))])
      means.append(torch.mean(prob_ratios * discounted_rewards))
    return torch.mean(torch.stack(means))

def surrogate_constraint(policy, new_policy, trajs):
    means = []
    for tau in trajs:
        states, _, rewards = tau
        discounted_rewards = GAMMA ** torch.arange(len(rewards.T))
        policy_probs = policy(states)
        #print(f"Policy probs shape: {policy_probs.shape}")
        new_policy_probs = new_policy(states)
        kl_divs = torch.sum(policy_probs * torch.log(policy_probs / new_policy_probs), axis=1)
        means.append(torch.mean(kl_divs * discounted_rewards))
    return torch.mean(torch.stack(means))

def flatten_params(policy):
    params = torch.cat([p.view(-1) for p in policy.parameters()])
    print(f"Params shape {params.shape}")
    return params

def linesearch(policy, new_policy, trajs):
    f_theta = surrogate_objective(policy, policy, trajs)
    while surrogate_constraint(policy, new_policy, trajs) > EPSILON or surrogate_objective(policy, new_policy, trajs) < f_theta:
        theta = flatten_params(policy)
        theta_new = flatten_params(new_policy)
        theta_new = theta + ALPHA * (theta_new - theta)
        n = 0
        for _, p in enumerate(new_policy.parameters()):
            num_elements = p.numel()
            p.data = theta_new[n:n+num_elements].view(p.shape)
            n += num_elements
    return new_policy

In [16]:
policy = CategoricalPolicy(8, 5).to(device)

for e in range(EPOCHS):
  trajs = sample_rollouts(policy, env, BATCH_SIZE)
  g_u_tau = [grad_U_tau(policy, tau) for tau in trajs]
  Fish = torch.stack([gut.unsqueeze(1) @ gut.unsqueeze(0) for gut in g_u_tau]).mean(axis=0)
  g_u = torch.stack(g_u_tau).mean(axis=0)
  u = torch.linalg.pinv(Fish) @ g_u.unsqueeze(0).T
  g = u * torch.sqrt(2 * EPSILON / (g_u @ u))
  #print(f"G: {g}")
  n = 0
  new_policy = CategoricalPolicy(8, 5).to(device)
  new_policy.load_state_dict(policy.state_dict())
  for i, p in enumerate(new_policy.parameters()):
    num_elements = p.numel()
    p.data += g[n:n+num_elements].view(p.shape)
    n += num_elements
  policy = linesearch(policy, new_policy, trajs)
  policy.zero_grad()
  if e % 10 == 0:
    print(f"iter {e}: {avg_batch_rewards(trajs)}")

iter 0: -8.163849246412052
